## Demo: Denoising of 3D mitochondrial outer membrane data

We extended the 2D Unet of SN2N to a 3D Unet to suit for three-dimensional XYZ data.  
In general, the core parameters of SN2N's 2D and 3D versions are essentially the same. While there are slight differences in specific parameters, they do not affect user usage. Users only need to grasp the core parameters to freely use SN2N for 2D and 3D data denoising tasks. For detailed parameters, please refer to SN2N.get_options.py.

<p>
<img src='./imgs/net.png' align="left" width=500>
</p>

## Raw data

The effectiveness of SN2N on 3D networks was validated using 3D OMM network imaging of live COS-7 cells labeled with Tom20-mCherry on SD-SIM sysytem.  
The raw data has a size of 101 * 1478 * 1137 pixels, with a pixel size of 38.23 nm.

<p>
<img src='./imgs/t3-raw-MAX_colored.png' align="left" width=200>
</p>

## Generating training data for SN2N

### Step 1: Data preprocessing

We first perform 3D volumetric <font color="red">percentaged normalization</font> on the data, along with optional background subtraction, to reduce pixel variations, eliminate hot pixels, and remove baseline signals.  
This is done to enhance the adaptability of the model to the input data.

In [1]:
import os
import tifffile
import numpy as np
from SN2N.utils import *

flage = 1
datapath_list = []
pmin = 30
pmax = 99.95
imgpath = 'C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data/'
save_path = ('C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per/')

os.makedirs(save_path, exist_ok=True)
for (root, dirs, files) in os.walk(imgpath):
    for j, Ufile in enumerate(files):
        img_path = os.path.join(root, Ufile)
        datapath_list.append(img_path)
          
l = len(datapath_list)
for ll in range(l):
    image_data = tifffile.imread(datapath_list[ll])
    [t, x, y] = image_data.shape
    image_data_new = normalize_percentage(x = image_data, pmin = pmin, pmax = pmax, axis=None, clip=True, eps=1e-20, dtype=np.float32)
    image_data_new2 = 255*image_data_new
    tifffile.imsave(('%s/%d.tif') %(save_path, flage), image_data_new2.astype('uint8'))
    flage = flage + 1
    
print("Data preprocessing is completed.")

D:\software\Anaconda\envs\pytorch_q\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data preprocessing is completed.


### Step 2: Define custom parameters.

We first need to use raw_data to generate SN2N data pairs.  
When generating data pairs, we need to first confirm and <font color="red">set important parameters</font> related to data generation.

-----Parameters------  
<font color="red">=====Important==========</font>  
- <span style="color:blue">img_path:</span>  
    Path of raw images to train.  
- <span style="color:blue">P2Pmode(0 ~ 3):</span>  
    Augmentation mode for Patch2Patch.  
    0: NONE;  
    1: Direct interchange in t;  
    2: Interchange in single frame;  
    3: Interchange in multiple frame but in different regions;  
    {default: 0}  
- <span style="color:blue">P2Pup:</span>  
    Increase the dataset to its (1 + P2Pup) times size.  
    {default: 0}  
- <span style="color:blue">BAmode(0 ~ 2)</span>  
    Basic augmentation mode.  
    0: NONE;  
    1: double the dataset with random rotate&flip;  
    2: eightfold the dataset with random rotate&flip;  
    {default: 0}  
- <span style="color:blue">SWsize</span>  
    Interval pixel of sliding window for generating image pathes.  
    {default: 64}  
    
<font color="red">======Other parameters do not require modification; for details, refer to SN2N.get_options.datagen3D.========</font> 

In [2]:
import sys
from SN2N.datagen import generator3D
from SN2N.get_options import datagen3D

img_path = 'C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per'
P2Pmode = '0'
P2Pup = '0'
BAmode = '0'
SWsize = '64' 
SWfilter = '15'


datagen3D_args = [
    '--img_path', img_path,
    '--P2Pmode', P2Pmode,
    '--P2Pup', P2Pup,
    '--BAmode', BAmode,
    '--SWsize', SWsize,
    '--SWfilter', SWfilter
]

args = datagen3D(datagen3D_args)
print("Arguments received:", args)

Arguments received: Namespace(BAmode=0, P2P_patch='64', P2Pmode=0, P2Pup=0, SWfilter=15, SWmode=1, SWsize=64, ifx2=True, img_path='C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per', inter_mode='Fourier', vol_patch='16,128,128')


### Step 3: Execute data generation.

The <font color="red">raw data</font> used for generating training data is stored in the <font color="red">'Path:/raw_data'</font> directory, and the generated <font color="red">training data pairs</font> are by default stored in the <font color="red">'Path:/datasets'</font>

In [3]:
d = generator3D(img_path=args.img_path, P2Pmode=args.P2Pmode, P2Pup=args.P2Pup, BAmode=args.BAmode, SWsize=args.SWsize, SWfilter=args.SWfilter)

d.execute()
print("Data generation completed.")

The path for the raw images used for training is located under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per
The training dataset is being saved under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data\datasets
For number 1 frame
Saving training images: 100
Saving training images: 200
Saving training images: 300
Saving training images: 400
Saving training images: 500
Saving training images: 600
Saving training images: 700
Saving training images: 800
Saving training images: 900
Saving training images: 1000
Saving training images: 1100
Saving training images: 1200
Saving training images: 1300
Saving training images: 1400
Saving training images: 1500
Data generation completed.


## Training data

We need to use training data pairs to train SN2N model.  
When training, we need to first confirm and <font color="red">set important parameters</font> related to trainers.

-----Parameters------  
<font color="red">=====Important==========</font>  
- <span style="color:blue">img_path:</span>  
    Path of raw images to train.  
- <span style="color:blue">sn2n_loss:</span>  
    Weight of self-constrained loss.  
    <span style="color:red">One of the most crucial parameters that needs adjustment， generally set to 0~2.</span>  
    A higher sn2n_loss weight implies relatively better denoising and more pronounced smoothing effects.  
    {default: 1}  
- <span style="color:blue">bs:</span>  
    Training batch size.  
    {default: 4}  
- <span style="color:blue">lr</span>  
    Learning rate.       
    {default: 2e-4}  
- <span style="color:blue">epochs</span>  
    Total number of training epochs.  
    {default: 20}  
    
<font color="red">======Other parameters do not require modification; for details, refer to SN2N.get_options.trainer3D.========</font>  

### Step 1: Define custom parameters.

In [4]:
from SN2N.trainer import net3D
from SN2N.get_options import trainer3D

img_path = 'C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per'
sn2n_loss = '0.5'
bs = '4'
lr = '2e-4'
epochs = '20'

trainer3D_args = [
    '--img_path', img_path,
    '--sn2n_loss', sn2n_loss,
    '--bs', bs,
    '--lr', lr,
    '--epochs', epochs
]

args = trainer3D(trainer3D_args)
print("Arguments received:", args)

Arguments received: Namespace(bs=4, epochs=20, if_alr=True, img_path='C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per', lr=0.0002, sn2n_loss=0.5, vol_patch='16,128,128')


### Step 2: Execute training.

The <font color="red">raw data</font> used for generating training data is stored in the <font color="red">'Path:/raw_data'</font> directory  
the <font color="red">models</font> are by default stored in the <font color="red">'Path:/models'</font>  
During the training process, we also performed real-time predictions of images to monitor the model training progress. 
For 3D tasks, to avoid memory overflow during image prediction, we utilized the (0:16, 0:256, 0:256) portion of the raw image for prediction.  
The <font color="red">images</font> results are by default stored in the <font color="red">'Path:/images'</font> directory.

In [5]:
sn2nunet = net3D(img_path = args.img_path, sn2n_loss = args.sn2n_loss, bs = args.bs, lr = args.lr, epochs = args.epochs)
sn2nunet.train()
print("Training completed.")

The path for the raw images used for training is located under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data/raw_data_per
The training dataset is being saved under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data\datasets
Models is being saved under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data\models
Training temporary prediction images is being saved under:
C:/Users/admin/Desktop/code-V0.2.91/examples/denoising3D/data\images
[Epoch 0/20] [Batch 0/398] [loss:23.932259] time:0:00:03.357620
[Epoch 0/20] [Batch 1/398] [loss:21.132091] time:0:00:04.272618
[Epoch 0/20] [Batch 2/398] [loss:20.687671] time:0:00:05.024204
[Epoch 0/20] [Batch 3/398] [loss:19.527408] time:0:00:05.761202
[Epoch 0/20] [Batch 4/398] [loss:17.685314] time:0:00:06.521200
[Epoch 0/20] [Batch 5/398] [loss:18.662812] time:0:00:07.270198
[Epoch 0/20] [Batch 6/398] [loss:18.715259] time:0:00:08.010216
[Epoch 0/20] [Batch 7/398] [loss:16.330944] time:0:00:08.754215
[Epoch 0/

[Epoch 0/20] [Batch 120/398] [loss:7.503371] time:0:01:33.142407
[Epoch 0/20] [Batch 121/398] [loss:7.389357] time:0:01:33.884948
[Epoch 0/20] [Batch 122/398] [loss:7.335427] time:0:01:34.628704
[Epoch 0/20] [Batch 123/398] [loss:7.415597] time:0:01:35.376703
[Epoch 0/20] [Batch 124/398] [loss:7.340150] time:0:01:36.119244
[Epoch 0/20] [Batch 125/398] [loss:7.145583] time:0:01:36.874243
[Epoch 0/20] [Batch 126/398] [loss:7.258495] time:0:01:37.621702
[Epoch 0/20] [Batch 127/398] [loss:7.257190] time:0:01:38.365275
[Epoch 0/20] [Batch 128/398] [loss:7.094040] time:0:01:39.109274
[Epoch 0/20] [Batch 129/398] [loss:7.369014] time:0:01:39.862809
[Epoch 0/20] [Batch 130/398] [loss:7.101434] time:0:01:40.602369
[Epoch 0/20] [Batch 131/398] [loss:7.062003] time:0:01:41.351368
[Epoch 0/20] [Batch 132/398] [loss:7.148928] time:0:01:42.098950
[Epoch 0/20] [Batch 133/398] [loss:7.084277] time:0:01:42.841462
[Epoch 0/20] [Batch 134/398] [loss:6.928095] time:0:01:43.587392
[Epoch 0/20] [Batch 135/3

[Epoch 0/20] [Batch 247/398] [loss:6.351527] time:0:03:07.978392
[Epoch 0/20] [Batch 248/398] [loss:6.210404] time:0:03:08.726426
[Epoch 0/20] [Batch 249/398] [loss:6.036736] time:0:03:09.467388
[Epoch 0/20] [Batch 250/398] [loss:6.114484] time:0:03:10.214565
[Epoch 0/20] [Batch 251/398] [loss:6.016451] time:0:03:10.964563
[Epoch 0/20] [Batch 252/398] [loss:6.385544] time:0:03:11.704562
[Epoch 0/20] [Batch 253/398] [loss:6.141185] time:0:03:12.454560
[Epoch 0/20] [Batch 254/398] [loss:6.292979] time:0:03:13.204114
[Epoch 0/20] [Batch 255/398] [loss:6.395590] time:0:03:13.950667
[Epoch 0/20] [Batch 256/398] [loss:6.250869] time:0:03:14.708409
[Epoch 0/20] [Batch 257/398] [loss:6.239044] time:0:03:15.454932
[Epoch 0/20] [Batch 258/398] [loss:6.159757] time:0:03:16.196540
[Epoch 0/20] [Batch 259/398] [loss:6.084519] time:0:03:16.950268
[Epoch 0/20] [Batch 260/398] [loss:6.025013] time:0:03:17.697030
[Epoch 0/20] [Batch 261/398] [loss:6.154418] time:0:03:18.451562
[Epoch 0/20] [Batch 262/3

[Epoch 0/20] [Batch 374/398] [loss:5.781785] time:0:04:42.933099
[Epoch 0/20] [Batch 375/398] [loss:5.668810] time:0:04:43.692211
[Epoch 0/20] [Batch 376/398] [loss:5.561194] time:0:04:44.437209
[Epoch 0/20] [Batch 377/398] [loss:5.696575] time:0:04:45.178168
[Epoch 0/20] [Batch 378/398] [loss:5.619349] time:0:04:45.927554
[Epoch 0/20] [Batch 379/398] [loss:5.933715] time:0:04:46.675102
[Epoch 0/20] [Batch 380/398] [loss:5.704213] time:0:04:47.427749
[Epoch 0/20] [Batch 381/398] [loss:5.625972] time:0:04:48.174493
[Epoch 0/20] [Batch 382/398] [loss:5.857256] time:0:04:48.920257
[Epoch 0/20] [Batch 383/398] [loss:5.641269] time:0:04:49.663255
[Epoch 0/20] [Batch 384/398] [loss:5.457854] time:0:04:50.410254
[Epoch 0/20] [Batch 385/398] [loss:5.896425] time:0:04:51.146795
[Epoch 0/20] [Batch 386/398] [loss:5.528818] time:0:04:51.888757
[Epoch 0/20] [Batch 387/398] [loss:5.748531] time:0:04:52.630307
[Epoch 0/20] [Batch 388/398] [loss:5.780752] time:0:04:53.365967
[Epoch 0/20] [Batch 389/3

[Epoch 1/20] [Batch 104/398] [loss:5.800390] time:0:06:18.545831
[Epoch 1/20] [Batch 105/398] [loss:5.528519] time:0:06:19.294959
[Epoch 1/20] [Batch 106/398] [loss:5.496304] time:0:06:20.036957
[Epoch 1/20] [Batch 107/398] [loss:5.963872] time:0:06:20.775047
[Epoch 1/20] [Batch 108/398] [loss:5.610783] time:0:06:21.519561
[Epoch 1/20] [Batch 109/398] [loss:5.658121] time:0:06:22.259240
[Epoch 1/20] [Batch 110/398] [loss:5.443371] time:0:06:23.001312
[Epoch 1/20] [Batch 111/398] [loss:5.499208] time:0:06:23.745312
[Epoch 1/20] [Batch 112/398] [loss:5.617848] time:0:06:24.487540
[Epoch 1/20] [Batch 113/398] [loss:5.711899] time:0:06:25.229136
[Epoch 1/20] [Batch 114/398] [loss:5.741892] time:0:06:25.966215
[Epoch 1/20] [Batch 115/398] [loss:5.564443] time:0:06:26.707213
[Epoch 1/20] [Batch 116/398] [loss:5.674786] time:0:06:27.455609
[Epoch 1/20] [Batch 117/398] [loss:5.488427] time:0:06:28.204135
[Epoch 1/20] [Batch 118/398] [loss:5.671665] time:0:06:28.949133
[Epoch 1/20] [Batch 119/3

...

[Epoch 18/20] [Batch 283/398] [loss:5.583642] time:1:32:43.465233
[Epoch 18/20] [Batch 284/398] [loss:5.843337] time:1:32:44.218232
[Epoch 18/20] [Batch 285/398] [loss:5.977777] time:1:32:44.959230
[Epoch 18/20] [Batch 286/398] [loss:5.638685] time:1:32:45.710227
[Epoch 18/20] [Batch 287/398] [loss:6.018533] time:1:32:46.457226
[Epoch 18/20] [Batch 288/398] [loss:5.880446] time:1:32:47.198097
[Epoch 18/20] [Batch 289/398] [loss:5.455294] time:1:32:47.940159
[Epoch 18/20] [Batch 290/398] [loss:5.411249] time:1:32:48.696093
[Epoch 18/20] [Batch 291/398] [loss:5.547819] time:1:32:49.442091
[Epoch 18/20] [Batch 292/398] [loss:5.702481] time:1:32:50.190910
[Epoch 18/20] [Batch 293/398] [loss:5.716956] time:1:32:50.948908
[Epoch 18/20] [Batch 294/398] [loss:5.615844] time:1:32:51.702907
[Epoch 18/20] [Batch 295/398] [loss:5.707598] time:1:32:52.448905
[Epoch 18/20] [Batch 296/398] [loss:5.444983] time:1:32:53.197865
[Epoch 18/20] [Batch 297/398] [loss:5.672564] time:1:32:53.946864
[Epoch 18/

[Epoch 19/20] [Batch 10/398] [loss:5.736577] time:1:34:16.945441
[Epoch 19/20] [Batch 11/398] [loss:5.704990] time:1:34:17.682439
[Epoch 19/20] [Batch 12/398] [loss:5.412769] time:1:34:18.427437
[Epoch 19/20] [Batch 13/398] [loss:5.271783] time:1:34:19.177541
[Epoch 19/20] [Batch 14/398] [loss:6.050955] time:1:34:19.925539
[Epoch 19/20] [Batch 15/398] [loss:5.827832] time:1:34:20.668538
[Epoch 19/20] [Batch 16/398] [loss:5.441178] time:1:34:21.414679
[Epoch 19/20] [Batch 17/398] [loss:5.873088] time:1:34:22.154424
[Epoch 19/20] [Batch 18/398] [loss:5.993871] time:1:34:22.894993
[Epoch 19/20] [Batch 19/398] [loss:5.620525] time:1:34:23.638991
[Epoch 19/20] [Batch 20/398] [loss:5.429415] time:1:34:24.377990
[Epoch 19/20] [Batch 21/398] [loss:5.503166] time:1:34:25.123933
[Epoch 19/20] [Batch 22/398] [loss:5.455472] time:1:34:25.861710
[Epoch 19/20] [Batch 23/398] [loss:5.802683] time:1:34:26.607241
[Epoch 19/20] [Batch 24/398] [loss:5.324929] time:1:34:27.348786
[Epoch 19/20] [Batch 25/3

[Epoch 19/20] [Batch 136/398] [loss:5.583045] time:1:35:50.939683
[Epoch 19/20] [Batch 137/398] [loss:5.457400] time:1:35:51.685640
[Epoch 19/20] [Batch 138/398] [loss:5.463539] time:1:35:52.427639
[Epoch 19/20] [Batch 139/398] [loss:5.666388] time:1:35:53.173637
[Epoch 19/20] [Batch 140/398] [loss:5.517361] time:1:35:53.923635
[Epoch 19/20] [Batch 141/398] [loss:5.354827] time:1:35:54.665484
[Epoch 19/20] [Batch 142/398] [loss:5.655665] time:1:35:55.418281
[Epoch 19/20] [Batch 143/398] [loss:5.644156] time:1:35:56.173280
[Epoch 19/20] [Batch 144/398] [loss:5.590334] time:1:35:56.917278
[Epoch 19/20] [Batch 145/398] [loss:5.742329] time:1:35:57.664276
[Epoch 19/20] [Batch 146/398] [loss:5.769500] time:1:35:58.416792
[Epoch 19/20] [Batch 147/398] [loss:5.739005] time:1:35:59.161791
[Epoch 19/20] [Batch 148/398] [loss:5.618853] time:1:35:59.913789
[Epoch 19/20] [Batch 149/398] [loss:5.382748] time:1:36:00.663787
[Epoch 19/20] [Batch 150/398] [loss:5.475941] time:1:36:01.412676
[Epoch 19/

[Epoch 19/20] [Batch 261/398] [loss:5.754383] time:1:37:24.265187
[Epoch 19/20] [Batch 262/398] [loss:5.548306] time:1:37:25.007186
[Epoch 19/20] [Batch 263/398] [loss:5.237207] time:1:37:25.758184
[Epoch 19/20] [Batch 264/398] [loss:5.486203] time:1:37:26.507182
[Epoch 19/20] [Batch 265/398] [loss:5.656008] time:1:37:27.246326
[Epoch 19/20] [Batch 266/398] [loss:5.675839] time:1:37:27.988848
[Epoch 19/20] [Batch 267/398] [loss:5.240287] time:1:37:28.732846
[Epoch 19/20] [Batch 268/398] [loss:5.300573] time:1:37:29.477874
[Epoch 19/20] [Batch 269/398] [loss:5.582637] time:1:37:30.236843
[Epoch 19/20] [Batch 270/398] [loss:5.970582] time:1:37:30.981813
[Epoch 19/20] [Batch 271/398] [loss:5.794455] time:1:37:31.726810
[Epoch 19/20] [Batch 272/398] [loss:5.570266] time:1:37:32.474809
[Epoch 19/20] [Batch 273/398] [loss:5.935454] time:1:37:33.224808
[Epoch 19/20] [Batch 274/398] [loss:5.369742] time:1:37:33.967654
[Epoch 19/20] [Batch 275/398] [loss:5.355974] time:1:37:34.709652
[Epoch 19/

[Epoch 19/20] [Batch 386/398] [loss:5.493003] time:1:38:57.509406
[Epoch 19/20] [Batch 387/398] [loss:5.540669] time:1:38:58.254405
[Epoch 19/20] [Batch 388/398] [loss:6.033198] time:1:38:58.992403
[Epoch 19/20] [Batch 389/398] [loss:5.298589] time:1:38:59.734726
[Epoch 19/20] [Batch 390/398] [loss:5.690325] time:1:39:00.483745
[Epoch 19/20] [Batch 391/398] [loss:5.679677] time:1:39:01.224743
[Epoch 19/20] [Batch 392/398] [loss:5.757166] time:1:39:01.976741
[Epoch 19/20] [Batch 393/398] [loss:5.342169] time:1:39:02.727739
[Epoch 19/20] [Batch 394/398] [loss:5.873445] time:1:39:03.474104
[Epoch 19/20] [Batch 395/398] [loss:5.594581] time:1:39:04.222102
[Epoch 19/20] [Batch 396/398] [loss:5.620078] time:1:39:04.973101
[Epoch 19/20] [Batch 397/398] [loss:5.498493] time:1:39:05.718099
Training completed.
